In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections

In [2]:
x_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터3] samp_cst_feat'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

df = pd.merge(df_x, df_y, on='cst_id_di')

In [3]:
df.MRC_ID_DI[df.MRC_ID_DI > 0 ] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [4]:
nc_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터4] variable_dtype'}.xlsx")
nc = pd.read_excel(nc_name, index_col=0)

In [5]:
X = df.drop(columns = ['MRC_ID_DI'], axis=1)
y = df['MRC_ID_DI']

In [6]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

def add_interactions(df):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis = 1)
    
    return df

X = add_interactions(X)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1, stratify = y)

In [8]:
test = X_test
test['cst_id_di'] = y_test.index
test = test.set_index('cst_id_di')
test['MRC_ID_DI'] = y_test

X_test = test.drop(columns = ['MRC_ID_DI'], axis=1)
y_test = test['MRC_ID_DI']



train = X_train
train['cst_id_di'] = y_train.index
train = train.set_index('cst_id_di')
train['MRC_ID_DI'] = y_train

train_0 = train[train['MRC_ID_DI'] == 0].sample(frac=1)
train_1 = train[train['MRC_ID_DI'] == 1].sample(frac=1)

sample_size = len(train_0) if len(train_0) < len(train_1) else len(train_1)

train_f = pd.concat([train_0.head(sample_size), train_1.head(sample_size)]).sample(frac=1)

X_train = train_f.drop(columns = ['MRC_ID_DI'], axis=1)
y_train = train_f['MRC_ID_DI']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [9]:
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=331)
selected_features = select.fit(X_train, y_train)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X_train.columns[i] for i in indices_selected]

X_train = X_train[colnames_selected]
X_test = X_test[colnames_selected]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: UserWarning: Features [ 1453  3776  5455 11480] are constant.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1, stratify = y_train)

In [11]:
X_train[X_train.columns] = (X_train[X_train.columns]+1)/2*255
X_test[X_test.columns] = (X_test[X_test.columns]+1)/2*255
X_val[X_val.columns] = (X_val[X_val.columns]+1)/2*255

In [ ]:
from PIL import Image
import numpy as np
for ind, k in enumerate(y_train.index):
    data = np.zeros((331, 331, 3), dtype=np.uint8)
    for i in range(331):
        for j in range(331):
            data[i, j] = [(X_train.iloc[ind, j] + X_train.iloc[ind, i])//2, 
                          (X_train.iloc[ind, j] + X_train.iloc[ind, i])//2, 
                          (X_train.iloc[ind, j] + X_train.iloc[ind, i])//2]
    img = Image.fromarray(data, 'RGB')
    
    if y_train.iloc[ind] == 0:
        img_name = os.path.join(os.getcwd(), 'image5', 'train', '0', str(k) + '.png')
    else:
        img_name = os.path.join(os.getcwd(), 'image5', 'train', '1', str(k) + '.png')
    img.save(img_name)

0 90000218963
1 90000318827
2 90000439495
3 90000163937
4 90000239064
5 90000175399
6 90000103038
7 90000395076
8 90000172786
9 90000384826
10 90000428439
11 90000104086
12 90000275758
13 90000229649
14 90000091226
15 90000249409
16 90000131603
17 90000459007
18 90000016535
19 90000296142
20 90000067138
21 90000437452
22 90000419894
23 90000447652
24 90000030838
25 90000223890
26 90000014384
27 90000442205
28 90000282652
29 90000286757
30 90000325522
31 90000032754
32 90000374698
33 90000012945
34 90000265395
35 90000112196
36 90000197295
37 90000316760
38 90000093733
39 90000003792
40 90000160427
41 90000280301
42 90000212366
43 90000405995
44 90000326862
45 90000276838
46 90000247683
47 90000361467
48 90000450588
49 90000098273
50 90000041980
51 90000458153
52 90000242689
53 90000095537
54 90000089777
55 90000255364
56 90000382075
57 90000158105
58 90000228018
59 90000309760
60 90000223330
61 90000100276
62 90000264865
63 90000000354
64 90000001372
65 90000301172
66 90000359565
67 90

In [ ]:
for ind, k in enumerate(y_test.index):
    data = np.zeros((331, 331, 3), dtype=np.uint8)
    for i in range(331):
        for j in range(331):
            data[i, j] = [(X_test.iloc[ind, j] + X_test.iloc[ind, i])//2, 
                          (X_test.iloc[ind, j] + X_test.iloc[ind, i])//2, 
                          (X_test.iloc[ind, j] + X_test.iloc[ind, i])//2]
    img = Image.fromarray(data, 'RGB')
    if y_test.iloc[ind] == 0:
        img_name = os.path.join(os.getcwd(), 'image5', 'test', '0', str(k) + '.png')
    else:
        img_name = os.path.join(os.getcwd(), 'image5', 'test', '1', str(k) + '.png')
    img.save(img_name)

In [ ]:
for ind, k in enumerate(y_val.index):
    data = np.zeros((331, 331, 3), dtype=np.uint8)
    for i in range(331):
        for j in range(331):
            data[i, j] = [(X_val.iloc[ind, j] + X_val.iloc[ind, i])//2, 
                          (X_val.iloc[ind, j] + X_val.iloc[ind, i])//2, 
                          (X_val.iloc[ind, j] + X_val.iloc[ind, i])//2]
    img = Image.fromarray(data, 'RGB')
    if y_val.iloc[ind] == 0:
        img_name = os.path.join(os.getcwd(), 'image5', 'val', '0', str(k) + '.png')
    else:
        img_name = os.path.join(os.getcwd(), 'image5', 'val', '1', str(k) + '.png')
    img.save(img_name)